# Emojis

Looking for words that co-occur with emojis in tweets  

Ling 583  
25 Jan 2018

In [1]:
import pandas as pd
import numpy as np
import ujson
import unicodedata

from cytoolz import concat

pd.set_option('precision', 2)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_colwidth', 250)

Gotta load the data!

In [2]:
df = pd.DataFrame(ujson.loads(r) for r in open('tweets.json'))

In [3]:
df

,created_at,favorite_count,favorited,filter_level,id_str,in_reply_to_screen_name,in_reply_to_status_id_str,in_reply_to_user_id_str,lang,place,possibly_sensitive,retweet_count,retweeted,search_region,source_clean,text,truncated,user
0,1418446457,0,False,medium,543629961978847232,None,None,None,en,"{'attributes': {}, 'bounding_box': {'coordinates': [[[-117.117116, 32.582128], [-117.117116, 32.685406], [-116.9278103, 32.685406], [-116.9278103, 32.582128]]], 'type': 'Polygon'}, 'country': 'United States', 'country_code': 'US', 'full_name': 'C...",False,0,False,San Diego region,Twitter for iPhone,This why I can't play 2k w yall 😭,False,"{'contributors_enabled': False, 'created_at': 'Tue Aug 23 23:07:00 +0000 2011', 'default_profile': True, 'default_profile_image': False, 'description': 'Cj White|Former Varsity Pearland DB | future civil engineer and buisness owner|', 'favourites..."
1,1418255902,0,False,medium,542830714937290752,None,None,None,en,"{'attributes': {}, 'bounding_box': {'coordinates': [[[-117.1244522, 32.64634], [-117.1244522, 32.69328], [-117.049103, 32.69328], [-117.049103, 32.64634]]], 'type': 'Polygon'}, 'country': 'United States', 'country_code': 'US', 'full_name': 'Natio...",False,0,False,San Diego region,Twitter for Android,mood: empty,False,"{'contributors_enabled': False, 'created_at': 'Tue Dec 17 00:24:18 +0000 2013', 'default_profile': False, 'default_profile_image': False, 'description': 'phora ♡', 'favourites_count': 1918, 'follow_request_sent': None, 'followers_count': 250, 'fo..."
2,1418017575,0,False,medium,541831098951430145,Rafamere,541831024011784192,2756341945,en,"{'attributes': {}, 'bounding_box': {'coordinates': [[[-117.117116, 32.582128], [-117.117116, 32.685406], [-116.9278103, 32.685406], [-116.9278103, 32.582128]]], 'type': 'Polygon'}, 'country': 'United States', 'country_code': 'US', 'full_name': 'C...",False,0,False,San Diego region,Twitter for iPhone,@Rafamere but I did ...,False,"{'contributors_enabled': False, 'created_at': 'Fri Nov 14 05:52:51 +0000 2014', 'default_profile': True, 'default_profile_image': False, 'description': 'SanDiego, California ig; seebastiann_', 'favourites_count': 373, 'follow_request_sent': None,..."
3,1418168139,0,False,medium,542462612336680960,None,None,None,en,"{'attributes': {}, 'bounding_box': {'coordinates': [[[-117.282538, 32.53962], [-117.282538, 33.0804044], [-116.9274403, 33.0804044], [-116.9274403, 32.53962]]], 'type': 'Polygon'}, 'country': 'United States', 'country_code': 'US', 'full_name': 'S...",False,0,False,San Diego region,Twitter for iPhone,"i continue to experience butterflies in my stomach when the stewardess says ""welcome to san diego."" #2ndHomeLove",False,"{'contributors_enabled': False, 'created_at': 'Wed Sep 02 21:26:51 +0000 2009', 'default_profile': False, 'default_profile_image': False, 'description': 'meaning-seeking, daughter lovin', people watching, God praising, light recording, 6' bohemia..."
4,1418366959,0,False,medium,543296523359244288,None,None,None,en,"{'attributes': {}, 'bounding_box': {'coordinates': [[[-124.482003, 32.528832], [-124.482003, 42.009519], [-114.131212, 42.009519], [-114.131212, 32.528832]]], 'type': 'Polygon'}, 'country': 'United States', 'country_code': 'US', 'full_name': 'Cal...",False,0,False,San Diego region,Twitter for Android,Nikka we made it! :))) http://t.co/WbK7XpSPsL,False,"{'contributors_enabled': False, 'created_at': 'Wed May 29 07:00:57 +0000 2013', 'default_profile': True, 'default_profile_image': False, 'description': None, 'favourites_count': 2515, 'follow_request_sent': None, 'followers_count': 183, 'followin..."
5,1418139650,0,False,medium,542343119123914752,None,None,None,en,"{'attributes': {}, 'bounding_box': {'coordinates': [[[-117.053546, 32.743581], [-117.053546, 32.7958], [-116.981714, 32.7958], [-116.981714, 32.743581]]], 'type': 'Polygon'}, 'country': 'United States', 'country_code': 'US', 'full_name': 'La Mesa...",False,0,False,San Diego region,Twitter for iPhone,I don't fuckin ca

Suppose we're interested in how the 😂 emoji is used on twitter.  We'll need to search for occurrences of 😂, which as a unicode character is no different from A or ॐ.  It can be hard to type, though, so we'll look it up by name in the Unicode character database:

In [4]:
emoji = unicodedata.lookup('face with tears of joy')
emoji

'😂'

Next, we'll normalize and tokenize the text just like we did with the CellarTracker data last week:

In [5]:
df['bow'] = df['text'].str.lower().str.replace(r'(\W|\d)+', ' ').str.split()

And count word occurrences:

In [6]:
subset = df['text'].str.contains(emoji)
f = pd.DataFrame({'all': pd.value_counts(list(concat(df['bow']))),
                  emoji: pd.value_counts(list(concat(df[subset]['bow'])))})

In [7]:
f.sort_values('all', ascending=False)

,all,😂
i,90521,5232.0
t,64351,3694.0
the,50910,2392.0
to,47683,2055.0
you,43147,2415.0
co,42777,2583.0
a,40714,1974.0
http,40457,2249.0
my,34092,2038.0
and,28590,1405.0


And finally calculate pointwise mutual information:

In [8]:
f['pmi'] = np.log2( (f[emoji] * np.sum(f['all'])) / 
                        (f['all'] * np.sum(f[emoji])) )
f[f['all']>25].sort_values('pmi',ascending=False)

,all,😂,pmi
bhui,120,119.0,4.23
hpgpc,120,119.0,4.23
onseek,29,28.0,4.19
nashnewvideo,130,119.0,4.11
laughed,229,165.0,3.76
nashsnewvideo,212,150.0,3.74
cried,254,153.0,3.51
funnypicsdepot,30,16.0,3.33
weloverobdyrdek,34,18.0,3.32
lmfaoooo,30,15.0,3.24


Onseek?  Bhui?  These are parts of usernames and URLs, and we probably don't want to count them as words. 

In [9]:
df['text'][df['text'].str.contains('onseek',case=False)]

10928     #NashsNewVideo 🐶 @Nashgrier #NashNewVideo \n\nI laughed so hard I cried 😂😂 yall should watch it ! \n\nhttp://t.co/P7Onseek5m 184
13182     #NashsNewVideo 🐶 @Nashgrier #NashNewVideo \n\nI laughed so hard I cried 😂😂 yall should watch it ! \n\nhttp://t.co/P7Onseek5m 159
17119     #NashsNewVideo 🐶 @Nashgrier #NashNewVideo \n\nI laughed so hard I cried 😂😂 yall should watch it ! \n\nhttp://t.co/P7Onseek5m 196
41260     #NashsNewVideo 🐶 @Nashgrier #NashNewVideo \n\nI laughed so hard I cried 😂😂 yall should watch it ! \n\nhttp://t.co/P7Onseek5m 173
74431     #NashsNewVideo 🐶 @Nashgrier #NashNewVideo \n\nI laughed so hard I cried 😂😂 yall should watch it ! \n\nhttp://t.co/P7Onseek5m 100
82681     #NashsNewVideo 🐶 @Nashgrier #NashNewVideo \n\nI laughed so hard I cried 😂😂 yall should watch it ! \n\nhttp://t.co/P7Onseek5m 153
86335     #NashsNewVideo 🐶 @Nashgrier #NashNewVideo \n\nI laughed so hard I cried 😂😂 yall should watch it ! \n\nhttp://t.co/P7Onseek5m 191
88445     #NashsNewVideo 🐶 

To strip this stuff out, we'll have to change our normalization/tokenization step to take into account peculiarities of twitter text:

In [10]:
df['bow'] = df['text'].str.lower() \
                      .str.replace(r'@\w+', ' ') \
                      .str.replace(r'http[^ ]+', ' ') \
                      .str.replace(r'(\W|\d)+', ' ') \
                      .str.split()

In [11]:
f = pd.DataFrame({'all': pd.value_counts(list(concat(df['bow']))),
                   emoji: pd.value_counts(list(concat(df[subset]['bow'])))})
f['pmi'] = np.log2( (f[emoji] * np.sum(f['all'])) / 
                        (f['all'] * np.sum(f[emoji])) )
f['pmi'][f['all']>25].sort_values(ascending=False)

nashnewvideo     4.16
laughed          3.81
nashsnewvideo    3.78
cried            3.55
lmfaoooo         3.28
bomedy           3.05
lmfaooo          2.83
hahha            2.75
funniest         2.67
pleaseeee        2.58
                 ... 
ا                 NaN
اذا               NaN
اللهم             NaN
او                NaN
ب                 NaN
غير               NaN
لكن               NaN
ツ                 NaN
笑                 NaN
ﾟ                 NaN
Name: pmi, Length: 5071, dtype: float64

Looking at a bigger part of the table, we've got good evidence that 😂 is mostly used to indicate laughter.

In [12]:
pd.set_option('display.max_rows', 50)
f['pmi'][f['all']>100].sort_values(ascending=False)

nashnewvideo                   4.16
laughed                        3.81
nashsnewvideo                  3.78
cried                          3.55
yall                           2.54
dying                          2.46
lmaoo                          2.25
laughing                       2.12
lmfao                          2.04
weak                           2.01
funny                          1.99
sa                             1.99
hilarious                      1.93
ko                             1.86
lmaooo                         1.81
laugh                          1.76
na                             1.75
drake                          1.72
hahahah                        1.70
hahahaha                       1.64
ako                            1.63
dancing                        1.63
lmao                           1.63
sc                             1.63
hop                            1.62
                               ... 
hola                          -4.63
academiakidscd              

-------

To analyze this text, we needed to specially adapt the normalization/tokenization step.  We'll often have to do this when the text we're working with has some unique quirks.  In this case, though, lots of people work on twitter text and there are existing tools to work with it.  In particular NLTK includes functions specifically work working with tweets:

In [13]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer(preserve_case=False, reduce_len=False, strip_handles=True)

In [14]:
df['text'][150], tokenizer.tokenize(df['text'][150])

("@BFTB_Chargers why can't you just be positive?",
 ['why', "can't", 'you', 'just', 'be', 'positive', '?'])

In [15]:
df['bow'] = df['text'].apply(tokenizer.tokenize)
f = pd.DataFrame({'all': pd.value_counts(list(concat(df['bow']))),
                   emoji: pd.value_counts(list(concat(df[subset]['bow'])))})
f['pmi'] = np.log2( (f[emoji] * np.sum(f['all'])) / 
                        (f['all'] * np.sum(f[emoji])) )

In [16]:
f['pmi'][f['all']>100].sort_values(ascending=False)

😂                         4.06
http://t.co/bhui2HPGPC    4.04
#nashnewvideo             3.93
laughed                   3.58
#nashsnewvideo            3.56
cried                     3.33
🐶                         2.88
🍕                         2.52
💀                         2.40
yall                      2.31
dying                     2.24
lmaoo                     2.02
laughing                  1.87
lmfao                     1.81
sa                        1.78
weak                      1.78
funny                     1.77
hilarious                 1.72
✋                         1.66
ko                        1.66
😭                         1.63
lmaooo                    1.58
laugh                     1.55
drake                     1.55
na                        1.53
                          ... 
midnight                   NaN
navy                       NaN
ocean                      NaN
positive                   NaN
pre-ordered                NaN
return                     NaN
ser     